# Convert given array as a sparse connected fcn

- Input node cannot be connected to the others
- But all output nodes should be connected to at least one hidden node or input node

In [1]:
import numpy as np
import torch
import torch.nn as nn

## Connection also represents its acitvation function
- 0 : not connected
- 1 : linear 
- 2 : ReLU
- 3 : Sigmoid
- and so on..

**this index can be changed**

In [2]:
# 0 : Not connected
# 1 : linear
# 2 : ReLU
# 3 : Sigmoid
activations = [None, None, nn.ReLU(), nn.Sigmoid()]

# References
- [Concatenate layer output with additional input data](https://discuss.pytorch.org/t/concatenate-layer-output-with-additional-input-data/20462)

In [3]:
class MatrixForWANN():
    
    def __init__(self, mat, in_dim, out_dim):
        # get when initialized
        self.mat = mat
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        #calculate
        self.num_hidden_nodes = self.mat.shape[1] - self.in_dim   
        
        #when matrix has hidden layer  
        if self.num_hidden_nodes == 1:  
            self.hidden_dim = [1] 
        elif self.num_hidden_nodes == 0:
            self.hidden_dim = []
        else:
            self.hidden_dim = self.get_hidden_dim()   
            
            
    def get_hidden_dim(self):
        in_dim = self.in_dim
        out_dim = self.out_dim
        mat_mask = self.mat
        
        hidden_dim_list = []
        start_col_idx = 0
        finish_col_idx = in_dim -1   
        
        while(True):
            
            if finish_col_idx >= mat_mask.shape[1]:   
                print(finish_col_idx)
                break  
            
            if ((mat_mask.shape[0] - sum(hidden_dim_list)) == out_dim):  #example4 해결
                 break  #지금 hidden dimension들 합이랑 output dim 합이 row길이랑 같으면 더이상 탐색 필요 x
            
            for i in range(sum(hidden_dim_list), len(mat_mask)): #이부분이상한데..?   
    
                #밑에처럼 하면 example 2에서 오류가 남.
                #skip connection에 대한 예외처리 해줘야 함   
    
                if(mat_mask[i,start_col_idx:(finish_col_idx + 1)].sum() == 0):   
                
                    hidden_dim = i - sum(hidden_dim_list)
                    hidden_dim_list += [hidden_dim]
                    start_col_idx = finish_col_idx + 1
                    finish_col_idx += hidden_dim   
                    break    
                    
        return hidden_dim_list

# Examples

## Example 1

![](img/example1_1.png)

![](img/example1_2.png)

In [4]:
mat1 = np.array([[0,2,0,0,2,0,0,0,0,0],
                [2,0,2,0,0,0,0,0,0,0],
                [0,2,0,2,0,0,0,0,0,0],
                [0,0,0,0,0,1,1,0,0,0],
                [0,0,0,0,0,0,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,3],
                [0,0,0,0,0,0,0,0,3,0]])
in_dim = 5
out_dim = 2
mat_wann1 = MatrixForWANN(mat1, in_dim, out_dim)

In [5]:
# first position represents row : FROM
mat1[0]

array([0, 2, 0, 0, 2, 0, 0, 0, 0, 0])

In [6]:
# second position represents column : TO
mat1[:, 0]

array([0, 2, 0, 0, 0, 0, 0])

In [7]:
# get destinations of input layer
mat1[:mat_wann1.num_hidden_nodes, :in_dim]

array([[0, 2, 0, 0, 2],
       [2, 0, 2, 0, 0],
       [0, 2, 0, 2, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [8]:
# num of total hidden nodes
mat_wann1.num_hidden_nodes

5

In [9]:
# get number of hidden layers and its node index
mat_wann1.hidden_dim

[3, 2]

## Example 2

![](img/example2_1.png)

![](img/example2_2.png)

In [10]:
mat2 = np.array([[2,0,0,0,0,0],
                [0,0,0,3,0,1]])
in_dim = 5
out_dim = 1
mat_wann2 = MatrixForWANN(mat2, in_dim, out_dim)

KeyboardInterrupt: 

In [11]:
mat_wann2.num_hidden_nodes

NameError: name 'mat_wann2' is not defined

In [12]:
mat_wann2.hidden_dim

NameError: name 'mat_wann2' is not defined

# Example 3

![](img/example3_1.png)

In [13]:
mat3 = np.array([[2,0,0,3,0],
                [0,0,0,0,1]])
in_dim = 5
out_dim = 2
mat_wann3 = MatrixForWANN(mat3, in_dim, out_dim)

In [14]:
mat_wann3.num_hidden_nodes

0

In [15]:
mat_wann3.hidden_dim

[]

## Example 4

![](img/example4_1.png)

In [16]:
mat4 = np.array([[3,0,1,0,0],
                [0,3,0,0,0],
                [0,0,0,2,0],
                [0,0,0,0,2],
                [0,0,1,2,0],
                [0,0,1,0,0]])
in_dim = 3
out_dim = 4
mat_wann4 = MatrixForWANN(mat4, in_dim, out_dim)

In [17]:
mat_wann4.num_hidden_nodes

2

In [18]:
mat_wann4.hidden_dim

[2, 3]

## Example 5
- Example 1에서 Input4 -> Hidden 5로 가는 Skip connection이 추가됨

![](img/example5_1.png)

In [19]:
mat5 = np.array([[0,2,0,0,2,0,0,0,0,0],
                [2,0,2,0,0,0,0,0,0,0],
                [0,2,0,2,0,0,0,0,0,0],
                [0,0,0,0,0,1,1,0,0,0],
                [0,0,0,2,0,0,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,3],
                [0,0,0,0,0,0,0,0,3,0]])
in_dim = 5
out_dim = 2
mat_wann5 = MatrixForWANN(mat5, in_dim, out_dim)

# class WANNFCN

In [20]:
def wrap_activation(x, idx_activation, activations) :  
    if idx_activation == 0 :
        assert True
    elif idx_activation == 1 :
        return nn.Linear(1,1)(x)
    else : 
        return activations[idx_activation](nn.Linear(1,1)(x))

In [ ]:
nn.Linear(784, 24)

In [127]:
# without hidden layer counts

class WANNFCN(nn.Module) : 
    def __init__(self, mat_wann, activations) : 
        super(WANNFCN, self).__init__()
        self.mat = mat_wann.mat
        self.in_dim = mat_wann.in_dim
        self.out_dim = mat_wann.out_dim
        self.num_hidden_nodes = mat_wann.num_hidden_nodes
        self.hidden_dim = mat_wann.hidden_dim
        
        self.activations = activations
        
        self.nodes = {}
        '''
        nodes라는 dictionary 안에 아래와 같이 저장됨
        'hidden_1' : 해당 노드
        'hidden_2' : 해당 노드
        ...
        'output_1' : 해당 output 노드, hidden node로부터 연결되어있음
        'output_2' : 해당 output 노드, input node, hidden node로부터 연결되어있음
        '''
        
    def forward(self, x) : 
        
        # hidden node가 한개라도 있을때
        if self.num_hidden_nodes != 0 :
            self.to_hidden(x)
        
        # output은 반드시 있음
        outputs = self.to_output(x)
        print(self.nodes)
        
        return outputs
    
    def to_output(self, x) :
        return ''
    
    def to_hidden(self, x) : 
        # input layer와 모든 이전 hidden layer를 탐색
        # 그렇지 않으면 skip connection을 놓칠수 있음
        # 모든 node와 connection은 dictionary self.nodes에 저장
        print(self.hidden_dim)
        hidden_node_counts = 0
        
        
        ############################### loop for hidden nodes
        for idx_hidden_row in list(range(0, self.mat.shape[0])) : 
            #connections_from_input = self.mat[idx_hidden_row, :self.in_dim]
            connections_from_input = self.mat[idx_hidden_row, :]
            print('connection from input : ', connections_from_input)
            if connections_from_input.sum() != 0 :
                count_connection = 0
                input_node = None
                ############################# loop for input nodes
                for idx_input_col, activation_type in enumerate(connections_from_input) :
                    print('idx_input_col %s, activation_type %s' % (idx_input_col, activation_type))
                    if activation_type != 0 and count_connection == 0:
                        # x[sample index, positional index for input]
                        print('\n**first input node')

                        # 1) idx_input_col 이 input에서 오는 경우
                        if idx_input_col < self.in_dim : 
                            input_node = wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations)
                        # 2) idx_input_col이 hidden에서 오는 경우
                        elif idx_input_col >= self.in_dim : 
                            input_node = wrap_activation(self.nodes['hidden_%d'%(idx_input_col-self.in_dim)], activation_type, activations)

                        print(input_node)
                        count_connection += 1
                    elif activation_type != 0 and count_connection != 0 :
                        print('%s input node' % idx_input_col)
                        # x[sample index, positional index for input]
                        # torch.sum returns the addition of two tensors

                        print('\n**input_node', input_node.shape)
                        print(input_node)

                        #new_node = wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations)

                        new_node = None
                        # 1) idx_input_col 이 input에서 오는 경우
                        if idx_input_col < self.in_dim : 
                            new_node = wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations)
                        # 2) idx_input_col이 hidden에서 오는 경우
                        elif idx_input_col >= self.in_dim : 
                            new_node = wrap_activation(self.nodes['hidden_%d'%(idx_input_col-self.in_dim)], activation_type, activations)



                        print('\n**wrap_activation', new_node.shape)
                        print(new_node)
                        input_node = input_node + new_node
                        print('\n**sum', input_node.shape)
                        print(input_node)


                        #input_node = torch.sum(input_node, wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations))
                        count_connection += 1
            # connect all input nodes to given hidden node
            if idx_hidden_row < self.num_hidden_nodes : 
                self.nodes['hidden_%d'%idx_hidden_row] = input_node 
            else : 
                self.nodes['output_%d'%(idx_hidden_row-self.num_hidden_nodes)] = input_node 
        # sum all numbers of hidden nodes from this layer
        hidden_node_counts += 1

            
            ############ 2. 두번째 이후의 layer일 경우 : input을 x로 받거나, 앞의 hidden node로 받름
            #if idx_hidden_layer == 0 :

In [128]:
model = WANNFCN(mat_wann1, activations)

numpy_input = np.array([[1,2,3,4,5],
                        [6,7,8,9,10],
                        [11,12,13,14,15]])

#numpy_input = np.array([[1,2,3,4,5]])
numpy_input = torch.from_numpy(numpy_input).float()
model(numpy_input)

[3, 2]
connection from input :  [0 2 0 0 2 0 0 0 0 0]
idx_input_col 0, activation_type 0
idx_input_col 1, activation_type 2

**first input node
tensor([[0.3565],
        [3.3295],
        [6.3025]], grad_fn=<ReluBackward0>)
idx_input_col 2, activation_type 0
idx_input_col 3, activation_type 0
idx_input_col 4, activation_type 2
4 input node

**input_node torch.Size([3, 1])
tensor([[0.3565],
        [3.3295],
        [6.3025]], grad_fn=<ReluBackward0>)

**wrap_activation torch.Size([3, 1])
tensor([[0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)

**sum torch.Size([3, 1])
tensor([[0.3565],
        [3.3295],
        [6.3025]], grad_fn=<AddBackward0>)
idx_input_col 5, activation_type 0
idx_input_col 6, activation_type 0
idx_input_col 7, activation_type 0
idx_input_col 8, activation_type 0
idx_input_col 9, activation_type 0
connection from input :  [2 0 2 0 0 0 0 0 0 0]
idx_input_col 0, activation_type 2

**first input node
tensor([[0.0000],
        [2.0731],
        [4.3955]], gr

''

In [115]:
# with hidden layer counts

class WANNFCN(nn.Module) : 
    def __init__(self, mat_wann, activations) : 
        super(WANNFCN, self).__init__()
        self.mat = mat_wann.mat
        self.in_dim = mat_wann.in_dim
        self.out_dim = mat_wann.out_dim
        self.num_hidden_nodes = mat_wann.num_hidden_nodes
        self.hidden_dim = mat_wann.hidden_dim
        
        self.activations = activations
        
        self.nodes = {}
        '''
        nodes라는 dictionary 안에 아래와 같이 저장됨
        'hidden_1' : 해당 노드
        'hidden_2' : 해당 노드
        ...
        'output_1' : 해당 output 노드, hidden node로부터 연결되어있음
        'output_2' : 해당 output 노드, input node, hidden node로부터 연결되어있음
        '''
        
    def forward(self, x) : 
        
        # hidden node가 한개라도 있을때
        if self.num_hidden_nodes != 0 :
            self.to_hidden(x)
        
        # output은 반드시 있음
        outputs = self.to_output(x)
        print(self.nodes)
        
        return outputs
    
    def to_output(self, x) :
        return ''
    
    def to_hidden(self, x) : 
        # input layer와 모든 이전 hidden layer를 탐색
        # 그렇지 않으면 skip connection을 놓칠수 있음
        # 모든 node와 connection은 dictionary self.nodes에 저장
        print(self.hidden_dim)
        hidden_node_counts = 0
        
        ########################## loop for hidden layers
        # self.hidden_dim : [3, 2]
        for idx_hidden_layer, num_hidden_nodes in enumerate(self.hidden_dim) : 
            print('idx_hidden_layer : %s, num_hidden_nodes : %s' % (idx_hidden_layer, num_hidden_nodes))
            
            ########### 1. 첫번째 레이어일 경우 : input을 x로 받음
            indices_hidden_row = []
            if idx_hidden_layer == 0 :
                indices_hidden_row = list(range(0, num_hidden_nodes))
            else :
                indices_hidden_row = list(range(hidden_node_counts, hidden_node_counts+num_hidden_nodes))
            print('indices_hidden_row : %s' % indices_hidden_row)

            ############################### loop for hidden nodes
            for idx_hidden_row in indices_hidden_row : 
                #connections_from_input = self.mat[idx_hidden_row, :self.in_dim]
                connections_from_input = self.mat[idx_hidden_row, :]
                print('connection from input : ', connections_from_input)
                if connections_from_input.sum() != 0 :
                    count_connection = 0
                    input_node = None
                    ############################# loop for input nodes
                    for idx_input_col, activation_type in enumerate(connections_from_input) :
                        print('idx_input_col %s, activation_type %s' % (idx_input_col, activation_type))
                        if activation_type != 0 and count_connection == 0:
                            # x[sample index, positional index for input]
                            print('\n**first input node')

                            # 1) idx_input_col 이 input에서 오는 경우
                            if idx_input_col < self.in_dim : 
                                input_node = wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations)
                            # 2) idx_input_col이 hidden에서 오는 경우
                            elif idx_input_col >= self.in_dim : 
                                input_node = wrap_activation(self.nodes['hidden_%d'%(idx_input_col-self.in_dim)], activation_type, activations)

                            print(input_node)
                            count_connection += 1
                        elif activation_type != 0 and count_connection != 0 :
                            print('%s input node' % idx_input_col)
                            # x[sample index, positional index for input]
                            # torch.sum returns the addition of two tensors

                            print('\n**input_node', input_node.shape)
                            print(input_node)

                            #new_node = wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations)

                            new_node = None
                            # 1) idx_input_col 이 input에서 오는 경우
                            if idx_input_col < self.in_dim : 
                                new_node = wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations)
                            # 2) idx_input_col이 hidden에서 오는 경우
                            elif idx_input_col >= self.in_dim : 
                                new_node = wrap_activation(self.nodes['hidden_%d'%(idx_input_col-self.in_dim)], activation_type, activations)



                            print('\n**wrap_activation', new_node.shape)
                            print(new_node)
                            input_node = input_node + new_node
                            print('\n**sum', input_node.shape)
                            print(input_node)


                            #input_node = torch.sum(input_node, wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations))
                            count_connection += 1
                # connect all input nodes to given hidden node
                self.nodes['hidden_%d'%idx_hidden_row] = input_node 

            # sum all numbers of hidden nodes from this layer
            hidden_node_counts += len(indices_hidden_row)

            
            ############ 2. 두번째 이후의 layer일 경우 : input을 x로 받거나, 앞의 hidden node로 받름
            #if idx_hidden_layer == 0 :

In [116]:
model = WANNFCN(mat_wann1, activations)

numpy_input = np.array([[1,2,3,4,5],
                        [6,7,8,9,10],
                        [11,12,13,14,15]])

#numpy_input = np.array([[1,2,3,4,5]])
numpy_input = torch.from_numpy(numpy_input).float()
model(numpy_input)

[3, 2]
idx_hidden_layer : 0, num_hidden_nodes : 3
indices_hidden_row : [0, 1, 2]
connection from input :  [0 2 0 0 2 0 0 0 0 0]
idx_input_col 0, activation_type 0
idx_input_col 1, activation_type 2

**first input node
tensor([[0.0000],
        [0.9916],
        [1.9985]], grad_fn=<ReluBackward0>)
idx_input_col 2, activation_type 0
idx_input_col 3, activation_type 0
idx_input_col 4, activation_type 2
4 input node

**input_node torch.Size([3, 1])
tensor([[0.0000],
        [0.9916],
        [1.9985]], grad_fn=<ReluBackward0>)

**wrap_activation torch.Size([3, 1])
tensor([[0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)

**sum torch.Size([3, 1])
tensor([[0.0000],
        [0.9916],
        [1.9985]], grad_fn=<AddBackward0>)
idx_input_col 5, activation_type 0
idx_input_col 6, activation_type 0
idx_input_col 7, activation_type 0
idx_input_col 8, activation_type 0
idx_input_col 9, activation_type 0
connection from input :  [2 0 2 0 0 0 0 0 0 0]
idx_input_col 0, activation_type 2

**

''

In [109]:
class WANNFCN(nn.Module) : 
    def __init__(self, mat_wann, activations) : 
        super(WANNFCN, self).__init__()
        self.mat = mat_wann.mat
        self.in_dim = mat_wann.in_dim
        self.out_dim = mat_wann.out_dim
        self.num_hidden_nodes = mat_wann.num_hidden_nodes
        self.hidden_dim = mat_wann.hidden_dim
        
        self.activations = activations
        
        self.nodes = {}
        '''
        nodes라는 dictionary 안에 아래와 같이 저장됨
        'hidden_1' : 해당 노드
        'hidden_2' : 해당 노드
        ...
        'output_1' : 해당 output 노드, hidden node로부터 연결되어있음
        'output_2' : 해당 output 노드, input node, hidden node로부터 연결되어있음
        '''
        
    def forward(self, x) : 
        
        # hidden node가 한개라도 있을때
        if self.num_hidden_nodes != 0 :
            self.to_hidden(x)
        
        # output은 반드시 있음
        outputs = self.to_output(x)
        print(self.nodes)
        
        return outputs
    
    def to_hidden(self, x) : 
        # input layer와 모든 이전 hidden layer를 탐색
        # 그렇지 않으면 skip connection을 놓칠수 있음
        # 모든 node와 connection은 dictionary self.nodes에 저장
        print(self.hidden_dim)
        hidden_node_counts = 0
        
        print(x.shape)
        
        ########################## loop for hidden layers
        for idx_hidden_layer, num_hidden_nodes in enumerate(self.hidden_dim) : 
            print('idx_hidden_layer : %s, num_hidden_nodes : %s' % (idx_hidden_layer, num_hidden_nodes))
            
            
            ############################# 1. First layer 
            # Check input layer only
            #if idx_hidden_layer == 0 :
            indices_hidden_row = []
            if idx_hidden_layer == 0 :
                indices_hidden_row = list(range(0, num_hidden_nodes))
            else :
                indices_hidden_row = list(range(hidden_node_counts, hidden_node_counts+num_hidden_nodes))
            print('indices_hidden_row : %s' % indices_hidden_row)

            ############################### loop for hidden nodes
            for idx_hidden_row in indices_hidden_row : 
                #connections_from_input = self.mat[idx_hidden_row, :self.in_dim]
                connections_from_input = self.mat[idx_hidden_row, :]
                print('connection from input : ', connections_from_input)
                if connections_from_input.sum() != 0 :
                    count_connection = 0
                    input_node = None
                    ############################# loop for input nodes
                    for idx_input_col, activation_type in enumerate(connections_from_input) :
                        print('idx_input_col %s, activation_type %s' % (idx_input_col, activation_type))
                        if activation_type != 0 and count_connection == 0:
                            # x[sample index, positional index for input]
                            print('\n**first input node')
                            input_node = wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations)
                            print(input_node)
                            count_connection += 1
                        elif activation_type != 0 and count_connection != 0 :
                            print('%s input node' % idx_input_col)
                            # x[sample index, positional index for input]
                            # torch.sum returns the addition of two tensors

                            print('\n**input_node', input_node.shape)
                            print(input_node)

                            new_node = wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations)
                            print('\n**wrap_activation', new_node.shape)
                            print(new_node)
                            input_node = input_node + new_node
                            print('\n**sum', input_node.shape)
                            print(input_node)


                            #input_node = torch.sum(input_node, wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations))
                            count_connection += 1
                # connect all input nodes to given hidden node
                self.nodes['hidden_%d'%idx_hidden_row] = input_node 

            # sum all numbers of hidden nodes from this layer
            hidden_node_counts += len(indices_hidden_row)
                
                
            '''
            ############################# 2. Other hidden layers
            # Check input layer, previous hidden layers
            else : 
                print('################ else')
                # hidden node index (전체 히든 노드 중에서)
                indices_hidden_row = list(range(hidden_node_counts, hidden_node_counts+num_hidden_nodes))
                print('indices_hidden_row : %s' % indices_hidden_row)
                
                ############################### loop for hidden nodes
                for idx_hidden_row in indices_hidden_row : 
                    #connections_from_input = self.mat[idx_hidden_row, :self.in_dim]
                    connections_from_input = self.mat[idx_hidden_row, :]
                    print('connection from input : ', connections_from_input)
                    
                    count_connection = 0
                    input_node = None
                    
                    #################### input으로부터 오는 연결
                    if connections_from_input.sum() != 0 :
                        
                        ############################# loop for input nodes
                        for idx_input_col, activation_type in enumerate(connections_from_input) :
                            print('idx_input_col %s, activation_type %s' % (idx_input_col, activation_type))
                            if activation_type != 0 and count_connection == 0:
                                # x[sample index, positional index for input]
                                print('\n**first input node')
                                input_node = wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations)
                                print(input_node)
                                count_connection += 1
                            elif activation_type != 0 and count_connection != 0 :
                                print('%s input node' % idx_input_col)
                                # x[sample index, positional index for input]
                                # torch.sum returns the addition of two tensors
                                
                                print('\n**input_node', input_node.shape)
                                print(input_node)
                                
                                new_node = wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations)
                                print('\n**wrap_activation', new_node.shape)
                                print(new_node)
                                input_node = input_node + new_node
                                print('\n**sum', input_node.shape)
                                print(input_node)
                                
                                
                                #input_node = torch.sum(input_node, wrap_activation(x[:, idx_input_col].view(-1, 1), activation_type, activations))
                                count_connection += 1  
                        
                    ###################### 앞의 hidden layer로부터 오는 연결
                    #print('############# 앞의 hidden layer로부터 오는 연결 ###########')
                    #print(self.hidden_dim, idx_hidden_layer)
                    #for idx_hidden_row in indices_hidden_row : 
                    #    connections_from_input = self.mat[idx_hidden_row, :self.in_dim]
                   
                        
                    # connect all input nodes to given hidden node
                    self.nodes['hidden_%d'%idx_hidden_row] = input_node 
                
                hidden_node_counts += len(indices_hidden_row)
            '''
            
        return
    
    def to_output(self, x) : 
        # input layer와 모든 이전 hidden layer를 탐색
        # 그렇지 않으면 skip connection을 놓칠수 있음
        # 모든 node와 connection은 dictionary self.nodes에 저장
        
        outputs = 'test'
        return outputs

In [110]:
torch.tensor([0., 0., 0.])
torch.tensor([2.6683, 5.7021, 8.7358])
torch.tensor([0., 0., 0.]) + torch.tensor([2.6683, 5.7021, 8.7358])

tensor([2.6683, 5.7021, 8.7358])

In [111]:
model = WANNFCN(mat_wann1, activations)

numpy_input = np.array([[1,2,3,4,5],
                        [6,7,8,9,10],
                        [11,12,13,14,15]])

#numpy_input = np.array([[1,2,3,4,5]])
numpy_input = torch.from_numpy(numpy_input).float()
model(numpy_input)

[3, 2]
torch.Size([3, 5])
idx_hidden_layer : 0, num_hidden_nodes : 3
indices_hidden_row : [0, 1, 2]
connection from input :  [0 2 0 0 2 0 0 0 0 0]
idx_input_col 0, activation_type 0
idx_input_col 1, activation_type 2

**first input node
tensor([[1.0502],
        [1.3514],
        [1.6527]], grad_fn=<ReluBackward0>)
idx_input_col 2, activation_type 0
idx_input_col 3, activation_type 0
idx_input_col 4, activation_type 2
4 input node

**input_node torch.Size([3, 1])
tensor([[1.0502],
        [1.3514],
        [1.6527]], grad_fn=<ReluBackward0>)

**wrap_activation torch.Size([3, 1])
tensor([[ 3.4772],
        [ 6.8423],
        [10.2074]], grad_fn=<ReluBackward0>)

**sum torch.Size([3, 1])
tensor([[ 4.5274],
        [ 8.1937],
        [11.8601]], grad_fn=<AddBackward0>)
idx_input_col 5, activation_type 0
idx_input_col 6, activation_type 0
idx_input_col 7, activation_type 0
idx_input_col 8, activation_type 0
idx_input_col 9, activation_type 0
connection from input :  [2 0 2 0 0 0 0 0 0 0]
i

IndexError: index 5 is out of bounds for dimension 1 with size 5

In [91]:
model = WANNFCN(mat_wann3, activations)
numpy_input = np.array([[1,2,3,4,5],
                        [6,7,8,9,10],
                        [11,12,13,14,15]])
numpy_input = torch.from_numpy(numpy_input).float()
model(numpy_input)

'test'

In [85]:
model = WANNFCN(mat_wann4, activations)
numpy_input = np.array([[1,2,3,4],
                        [6,7,8,9],
                        [11,12,13,14]])
numpy_input = torch.from_numpy(numpy_input).float()
model(numpy_input)

[2, 3]
idx_hidden_layer : 0, num_hidden_nodes : 2
idx_hidden_row : [0, 1]
[[3 0 1]
 [0 3 0]]
idx_hidden_layer : 1, num_hidden_nodes : 3
idx_hidden_row : [2, 3, 4]


'test'

In [86]:
model = WANNFCN(mat_wann5, activations)
numpy_input = np.array([[1,2,3,4,5],
                        [6,7,8,9,10],
                        [11,12,13,14,15]])
numpy_input = torch.from_numpy(numpy_input).float()
model(numpy_input)

[3, 2]
idx_hidden_layer : 0, num_hidden_nodes : 3
idx_hidden_row : [0, 1, 2]
[[0 2 0 0 2]
 [2 0 2 0 0]
 [0 2 0 2 0]]
idx_hidden_layer : 1, num_hidden_nodes : 2
idx_hidden_row : [3, 4]


'test'